In [1]:
import numpy as np
import math
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.figsize'] = (10.0, 10.0)
np.set_printoptions(suppress=True) 

In [2]:
data = loadmat('./ex4data1.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [3]:
X = data['X']
y = data['y']
# convert the labels to a 10-d vector
y = pd.get_dummies( y.ravel() ).to_numpy() # 5000x10
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

Shape of X:  (5000, 400)
Shape of y:  (5000, 10)


## Model presentation

In [4]:
weights_data = loadmat('./ex4weights.mat')
weights_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

In [5]:
theta1, theta2 = weights_data['Theta1'], weights_data['Theta2']
print( theta1.shape, theta2.shape)

(25, 401) (10, 26)


## Feedforward and cost functon

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
def randInitializeWeights(L_in, L_out):
    eps = 0.12
    return 2 * eps * np.random.random_sample((L_out, L_in + 1)) - eps

In [8]:
def sigmoi_gradient(z):
    return sigmoid(z)*( 1 - sigmoid(z) )

In [9]:
def nn_cost_function_backprop(theta_params, X, y, lamda ):
    
    theta1, theta2 = np.split(theta_params,[25*401])
    theta1 = np.reshape(theta1,(25,401))
    theta2 = np.reshape(theta2,(10,26))
    
    m = X.shape[0]
    a1 = np.c_[np.ones(X.shape[0] ),X]     #5000x401
    
    z2 = theta1.dot(a1.T)
    a2 = np.r_[ np.ones( (1, z2.shape[1]) ), sigmoid(z2) ]
    
    z3 = theta2.dot( a2 )
    a3 = sigmoid(z3) # 10x5000
    
    J = (-1/m) * np.trace( np.log(a3).dot(y) + np.log(1 - a3).dot( ( 1 - y ))) + \
            ( lamda/(2*m) ) * ( np.trace( (theta1[:,1:]).T.dot(theta1[:,1:]) ) + np.trace( (theta2[:,1:]).T.dot(theta2[:,1:])))
    
    #Backpropagation
    delta3 = a3.T - y # (5000x10)
    delta2 = theta2[:, 1:].T.dot(delta3.T) * sigmoi_gradient(z2) # (25x5000)

    D1 = delta2.dot(a1) #(25x401)
    D2 = delta3.T.dot(a2.T) #(10x26)

    theta1_reg = np.c_[np.zeros((theta1.shape[0],1)),theta1[:,1:]]
    theta2_reg = np.c_[np.zeros((theta2.shape[0],1)),theta2[:,1:]]
    
    theta1_grad = D1/m + (theta1_reg * lamda)/m 
    theta2_grad= D2/m + (theta2_reg * lamda)/m
    
    grad = np.r_[theta1_grad.ravel(), theta2_grad.ravel()]
    return J, grad

In [10]:
theta_params = np.r_[theta1.ravel(), theta2.ravel()]
J, grad = nn_cost_function_backprop(theta_params, X, y, 1)
print(J)

0.3837698590909237


## Backpropagation

In [11]:
theta1_0 = randInitializeWeights(400, 25)
theta2_0 = randInitializeWeights(25, 10)
theta_0 = np.r_[theta1_0.ravel(), theta2_0.ravel()]

In [12]:
from scipy.optimize import minimize
lamda = 1
nn = minimize( fun = nn_cost_function_backprop, x0 = theta_0, 
              args = (X, y, lamda), 
              method = 'CG', jac = True, options = {'maxiter' : 400} )

In [13]:
nn

     fun: 0.3177414801332822
     jac: array([-0.00003294, -0.00000003,  0.00000003, ..., -0.000027  ,
       -0.0000709 , -0.00009441])
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 889
     nit: 400
    njev: 889
  status: 1
 success: False
       x: array([ 0.94942422, -0.00012581,  0.00013312, ..., -1.68210294,
        2.17810025,  0.29023886])

In [14]:
def predict( theta1, theta2, X ):
    z2 = theta1.dot(np.c_[np.ones(X.shape[0] ),X].T)
    a2 = np.r_[ np.ones( (1, z2.shape[1]) ), sigmoid(z2) ].T
    prob = sigmoid(a2.dot(theta2.T))
    pred =  np.argmax(prob, axis = 1 )+1
    return pred

In [15]:
y

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [16]:
res_layer1 = nn.x[0:25*401].reshape(25, 401)
res_layer2 = nn.x[25*401:].reshape(10, 26)
training_accuracy = np.mean( (data["y"].ravel() == predict( res_layer1, res_layer2, X ))*100 )
print( 'training accuracy = ', training_accuracy )

training accuracy =  99.5


## Gradient Checking

In [17]:
def nn_cost_function(theta_params, X, y, lamda ):
    
    theta1, theta2 = np.split(theta_params,[5*4])
    theta1 = np.reshape(theta1,(5, 4))
    theta2 = np.reshape(theta2,(3,6))
    
    m = X.shape[0]
    a1 = np.c_[np.ones(X.shape[0] ),X]
    
    z2 = theta1.dot(a1.T)
    a2 = np.r_[ np.ones( (1, z2.shape[1]) ), sigmoid(z2) ]
    
    z3 = theta2.dot( a2 )
    a3 = sigmoid(z3)
    
    J = (-1/m) * np.trace( np.log(a3).dot(y) + np.log(1 - a3).dot( ( 1 - y ))) + \
            ( lamda/(2*m) ) * ( np.trace( (theta1[:,1:]).T.dot(theta1[:,1:]) ) + np.trace( (theta2[:,1:]).T.dot(theta2[:,1:])))
    return J

In [18]:
def numerical_gradient(theta, X, y, lamda):
    num_grad = np.zeros(theta.shape[0])
    eps_vec = np.zeros(theta.shape[0])
    eps = 1e-4
    for i in range(theta.size):
        eps_vec[i] = eps
        loss1 = nn_cost_function(theta-eps_vec, X, y, lamda)
        loss2 = nn_cost_function(theta+eps_vec, X, y, lamda)
        
        #Compute grad
        num_grad[i] = (loss2 - loss1)/(2*eps)
        eps_vec[i] = 0.0
    return num_grad

In [19]:
input_layer_size = 3
hidden_layer_size = 5
num_labels = 3
m = 5

In [20]:
from sklearn import datasets, preprocessing

In [21]:
X, y = datasets.make_classification(n_samples=m, n_features=input_layer_size,n_informative=input_layer_size, n_redundant=0, n_repeated=0, n_classes=3)
y = pd.get_dummies( y.ravel() ).to_numpy()    #(5, 3)
print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  (5, 3)
y shape:  (5, 3)


In [22]:
theta1_0 = randInitializeWeights(input_layer_size, hidden_layer_size)
theta2_0 = randInitializeWeights(hidden_layer_size, num_labels)
theta_0 = np.r_[theta1_0.ravel(), theta2_0.ravel()]
theta_0.shape

(38,)

In [23]:
theta1_0.shape

(5, 4)

In [24]:
def nn_cost_function_backprop(theta_params, X, y, lamda ):
    
    theta1, theta2 = np.split(theta_params,[5*4])
    theta1 = np.reshape(theta1,(5, 4))
    theta2 = np.reshape(theta2,(3,6))
    
    m = X.shape[0]
    a1 = np.c_[np.ones(X.shape[0] ),X]     #5000x401
    
    z2 = theta1.dot(a1.T)
    a2 = np.r_[ np.ones( (1, z2.shape[1]) ), sigmoid(z2) ]
    
    z3 = theta2.dot( a2 )
    a3 = sigmoid(z3) # 10x5000
    
    J = (-1/m) * np.trace( np.log(a3).dot(y) + np.log(1 - a3).dot( ( 1 - y ))) + \
            ( lamda/(2*m) ) * ( np.trace( (theta1[:,1:]).T.dot(theta1[:,1:]) ) + np.trace( (theta2[:,1:]).T.dot(theta2[:,1:])))
    
    #Backpropagation
    delta3 = a3.T - y # (5000x10)
    delta2 = theta2[:, 1:].T.dot(delta3.T) * sigmoi_gradient(z2) # (25x5000)

    D1 = delta2.dot(a1) #(25x401)
    D2 = delta3.T.dot(a2.T) #(10x26)

    theta1_reg = np.c_[np.zeros((theta1.shape[0],1)),theta1[:,1:]]
    theta2_reg = np.c_[np.zeros((theta2.shape[0],1)),theta2[:,1:]]
    
    theta1_grad = D1/m + (theta1_reg * lamda)/m 
    theta2_grad= D2/m + (theta2_reg * lamda)/m
    
    grad = np.r_[theta1_grad.ravel(), theta2_grad.ravel()]
    return J, grad

In [25]:
J, grad = nn_cost_function_backprop(theta_0.copy(), X, y, 1)
print(grad.shape)

(38,)


In [26]:
numgrad = numerical_gradient(theta_0.copy(), X, y, 1)
print(numgrad.shape)

(38,)


In [27]:
diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)

In [28]:
diff

7.635234518506442e-11

In [29]:
numgrad

array([ 0.00414413,  0.01677476, -0.01372954,  0.02211303,  0.00336282,
       -0.00037137,  0.00102637, -0.01054504,  0.00283375,  0.00137006,
       -0.01656496,  0.0178358 , -0.00074517, -0.04039521,  0.01814783,
        0.02698411,  0.00289515,  0.00357534,  0.01343472,  0.02307375,
        0.08658999,  0.04232545,  0.02554712,  0.05014558,  0.06138933,
        0.05690897,  0.14643314,  0.08327513,  0.08086097,  0.07794555,
        0.11771219,  0.08657104,  0.31237821,  0.17619996,  0.15985427,
        0.16415111,  0.13575489,  0.14861195])

In [30]:
grad

array([ 0.00414413,  0.01677476, -0.01372954,  0.02211303,  0.00336282,
       -0.00037137,  0.00102637, -0.01054504,  0.00283375,  0.00137006,
       -0.01656496,  0.0178358 , -0.00074517, -0.04039521,  0.01814783,
        0.02698411,  0.00289515,  0.00357534,  0.01343472,  0.02307375,
        0.08658999,  0.04232545,  0.02554712,  0.05014558,  0.06138933,
        0.05690897,  0.14643314,  0.08327513,  0.08086097,  0.07794555,
        0.11771219,  0.08657104,  0.31237821,  0.17619996,  0.15985427,
        0.16415111,  0.13575489,  0.14861195])